✅ Step 1: Combine the sheets into a single decision matrix
We'll match provinces and extract only the relevant values.

In [3]:
# Step 1: Install Required Libraries
!pip install pandas numpy openpyxl scikit-learn

# Step 2: Upload the Excel file
from google.colab import files
uploaded = files.upload()

# Step 3: Load the Excel file
import pandas as pd

file_name = next(iter(uploaded))  # Get the uploaded file name
xls = pd.ExcelFile(file_name)

# Step 4: Read and Clean All Sheets
sheets_to_combine = [
    'wind speed',
    'sunshine',
    'air pollution   no2',
    'electricity deficit',
    'population ',
    'cost of barren land'
]

data_frames = {sheet: xls.parse(sheet) for sheet in sheets_to_combine}

# Rename columns for consistency
df_wind = data_frames['wind speed'].rename(columns={'wind speed m/s': 'Wind Speed'})
df_sun = data_frames['sunshine'].rename(columns={'Hourly solar radiation levels throughout the year 2022': 'Sunshine'})
df_pollution = data_frames['air pollution   no2'].rename(columns={'(micromol /Square meter)': 'NO2'})
df_electricity = data_frames['electricity deficit'].rename(columns={'MWh': 'Electricity Deficit'})
df_population = data_frames['population '].rename(columns={'population': 'Population'})
df_land_cost = data_frames['cost of barren land'].rename(columns={'(USD/m²) ': 'Land Cost'})

# Step 5: Merge into one decision matrix
df_combined = df_wind[['31 Provinces of Iran', 'Wind Speed']] \
    .merge(df_sun[['31 Provinces of Iran', 'Sunshine']], on='31 Provinces of Iran') \
    .merge(df_pollution[['31 Provinces of Iran', 'NO2']], on='31 Provinces of Iran') \
    .merge(df_electricity[['31 Provinces of Iran', 'Electricity Deficit']], on='31 Provinces of Iran') \
    .merge(df_population[['31 Provinces of Iran', 'Population']], on='31 Provinces of Iran') \
    .merge(df_land_cost[['31 Provinces of Iran', 'Land Cost']], on='31 Provinces of Iran')

# Step 6: Apply TOPSIS
from sklearn.preprocessing import MinMaxScaler
import numpy as np

# Extract criteria matrix
criteria_matrix = df_combined.iloc[:, 1:].values.astype(float)

# Step 6.1: Normalize the data
scaler = MinMaxScaler()
normalized_matrix = scaler.fit_transform(criteria_matrix)

# Step 6.2: Define weights (equal weight) and impacts (+ for beneficial, - for cost criteria)
weights = np.array([1, 1, 1, 1, 1, 1])  # You can customize weights
impacts = ['+', '+', '-', '-', '-', '-']  # Define whether each criterion is beneficial or cost

# Step 6.3: Apply weights
weighted_matrix = normalized_matrix * weights

# Step 6.4: Identify ideal and negative-ideal solutions
ideal = np.max(weighted_matrix, axis=0) * [1 if i == '+' else 0 for i in impacts] + \
        np.min(weighted_matrix, axis=0) * [1 if i == '-' else 0 for i in impacts]
negative_ideal = np.min(weighted_matrix, axis=0) * [1 if i == '+' else 0 for i in impacts] + \
                 np.max(weighted_matrix, axis=0) * [1 if i == '-' else 0 for i in impacts]

# Step 6.5: Calculate distances to ideal and negative-ideal
dist_ideal = np.linalg.norm(weighted_matrix - ideal, axis=1)
dist_negative = np.linalg.norm(weighted_matrix - negative_ideal, axis=1)

# Step 6.6: Calculate TOPSIS score and rank
topsis_score = dist_negative / (dist_ideal + dist_negative)
df_combined['TOPSIS Score'] = topsis_score
df_combined['Rank'] = df_combined['TOPSIS Score'].rank(ascending=False).astype(int)

# Step 7: Show Results
df_combined.sort_values(by='Rank').reset_index(drop=True)


Saving data.xlsx to data (1).xlsx


,31 Provinces of Iran,Wind Speed,Sunshine,NO2,Electricity Deficit,Population,Land Cost,TOPSIS Score,Rank
0,South Khorasan Province,14.4,3333.6,62.095,300,768898,0.2000,0.896391,1
1,Hormozgan,12.4,3208.2,72.381,250,1776415,0.6300,0.851427,2
2,Kohgiluyeh and Boyer-Ahmad,12.1,2825.9,63.836,60,713052,0.3000,0.833230,3
3,Kerman,11.3,3339.2,66.155,200,3164718,0.5400,0.822425,4
4,Bushehr,11.0,3210.6,71.108,120,1163400,0.8900,0.821428,5
5,Ardabil,13.9,2628.6,76.398,100,1270420,0.3300,0.816664,6
6,Chahar Mahal and Bakhtiari,10.6,2954.2,63.868,80,947763,0.0342,0.811095,7
7,North Khorasan Province,13.3,2956.1,140.094,400,863092,0.3080,0.808580,8
8,Zanjan,10.8,2894.6,69.718,50,1057461,0.5000,0.802353,9
9,Semnan,10.0,3116.6,126.817,70,702360,0.4200,0.795962,10


from matplotlib import pyplot as plt
_df_0['Wind Speed'].plot(kind='hist', bins=20, title='Wind Speed')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_1['Sunshine'].plot(kind='hist', bins=20, title='Sunshine')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_2['NO2'].plot(kind='hist', bins=20, title='NO2')
plt.gca().spines[['top', 'right',]].set_visible(False)

...

from matplotlib import pyplot as plt
import seaborn as sns
def _plot_series(series, series_name, series_index=0):
  palette = list(sns.palettes.mpl_palette('Dark2'))
  xs = series['Rank']
  ys = series['Electricity Deficit']
  
  plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

fig, ax = plt.subplots(figsize=(10, 5.2), layout='constrained')
df_sorted = _df_11.sort_values('Rank', ascending=True)
_plot_series(df_sorted, '')
sns.despine(fig=fig, ax=ax)
plt.xlabel('Rank')
_ = plt.ylabel('Electricity Deficit')

from matplotlib import pyplot as plt
_df_12['Wind Speed'].plot(kind='line', figsize=(8, 4), title='Wind Speed')
plt.gca().spines[['top', 'right']].set_visible(False)

from matplotlib import pyplot as plt
_df_13['Sunshine'].plot(kind='line', figsize=(8, 4), title='Sunshine')
plt.gca().spines[['top', 'right']].set_visible(False)

from matplotlib import pyplot as plt
_df_14['NO2'].plot(kind='line', figsize=(8, 4), title='NO2')
plt.gca().spines[['top', 'right']].set_visible(False)

from matplotlib import pyplot as plt
_df_15['Electricity Deficit'].plot(kind='line', figsize=(8, 4), title='Electricity Deficit')
plt.gca().spines[['top', 'right']].set_visible(False)